In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import geopy
from geopy.distance import geodesic
from geovoronoi import voronoi_regions_from_coords
from sklearn.cluster import KMeans, DBSCAN, OPTICS, MeanShift
import datetime
import folium
from tqdm import tqdm

### Data representation
$\begin{align}\{p^u_{i}|i=1,2,3,...,n ,\end{align}$
and
$\begin{align}u=1,2,3,...,U\}\end{align}$

* $\begin{align}p^u_{i}=(x_{i},y_{i},t_{i1},t_{i2})\end{align}$
* $\begin{align}x_{i},y_{i}:\end{align}$geographical coordinates
* $\begin{align}t_{i1},t_{i2}:\end{align}$start and end time

In [2]:
DATA_DIR = './data'

stop_df = pd.read_csv(DATA_DIR + '/pre_stop_points.csv')
stop_df.activity = stop_df.pop('activity_class')
stop_df.head()

,uid,id,lat,lng,start_time,end_time,activity
0,9497,250460,37.554742,127.026641,2019-11-07 10:45:58,2019-11-07 16:00:49,0.0
1,9497,240029,37.564022,127.035580,2019-11-06 23:27:51,2019-11-07 10:45:58,3.0
2,9497,238990,37.560267,127.033065,2019-11-06 20:22:16,2019-11-06 23:25:41,3.0
3,9497,234786,37.561929,127.038133,2019-11-06 17:15:56,2019-11-06 18:13:46,3.0
4,9497,226233,37.561171,127.037129,2019-11-06 07:49:15,2019-11-06 08:29:32,3.0


### Data quantization
$\begin{align}q_{i}(\end{align}$quantized version of$\begin{align}p_{i})=
\{c_{i},\mathcal{S}_{i},a_{i}\}\end{align}$

* $\begin{align}(x_{i},y_{i})\mapsto c_{i}\end{align}$
* $\begin{align}(t_{i1},t_{i2})\mapsto \mathcal{S}_{i}\end{align}$
* $\begin{align}a_{i}:\end{align}$activity classes

#### Spatial cell

* The location $\begin{align}(x_{i},y_{i})\mapsto\end{align}$a cell $\begin{align}c_{i}\end{align}$

In [3]:
def df_to_gdf(df, x, y):
    geometry = [Point(xy) for xy in zip(df[x], df[y])] # create Geometry series with lat / longitude
    df = df.drop([x, y], axis=1)
    gdf = gpd.GeoDataFrame(df, crs=None, geometry=geometry)
    return gdf

In [4]:
stop_gdf = df_to_gdf(stop_df, x='lng', y='lat')
stop_gdf.head()

,uid,id,start_time,end_time,activity,geometry
0,9497,250460,2019-11-07 10:45:58,2019-11-07 16:00:49,0.0,POINT (127.026641 37.55474155)
1,9497,240029,2019-11-06 23:27:51,2019-11-07 10:45:58,3.0,POINT (127.0355804 37.56402230000001)
2,9497,238990,2019-11-06 20:22:16,2019-11-06 23:25:41,3.0,POINT (127.0330645 37.5602669)
3,9497,234786,2019-11-06 17:15:56,2019-11-06 18:13:46,3.0,POINT (127.0381327 37.5619293)
4,9497,226233,2019-11-06 07:49:15,2019-11-06 08:29:32,3.0,POINT (127.03712865 37.56117105)


* clustering (e.g., k-means, DBSCAN, OPTICS, Mean shift)

In [5]:
def get_KMeans_cluster_centers(x, k):
    X = x.to_numpy()
    kmeans = KMeans(n_clusters=K, random_state=0).fit(X)
    coords = kmeans.cluster_centers_
    return coords

In [6]:
def get_cluster_centers(clusters):
    coords = list()
    
    for i in clusters[clusters.cluster != -1].cluster.unique():
        center = clusters[clusters.cluster == i].unary_union.centroid.coords[0]
        coord = list(center)
        coords.append(coord)
    return np.array(coords)

In [7]:
def get_DBSCAN_cluster_centers(x, min_samples, eps=1e-3):
    X = x.to_numpy()
    dbscan = DBSCAN(eps, min_samples).fit(X)
    gdf = df_to_gdf(x, x='lng', y='lat')
    gdf['cluster'] = dbscan.labels_
    coords = get_cluster_centers(clusters=gdf)
    return coords

In [8]:
def get_OPTICS_cluster_centers(x, min_samples):
    np.seterr(divide='ignore', invalid='ignore')
    X = x.to_numpy()
    optics = OPTICS(min_samples).fit(X)
    gdf = df_to_gdf(x, x='lng', y='lat')
    gdf['cluster'] = optics.labels_
    coords = get_cluster_centers(clusters=gdf)
    return coords

In [9]:
def get_MeanShift_cluster_centers(x, meters):
    X = x.to_numpy()
    
    origin = geopy.Point(x.lat[0], x.lng[0])
    destination = geodesic(meters=meters).destination(origin, 90)
    
    _x = np.array((x.lat[0], x.lng[0]))
    _y = np.array((destination.latitude, destination.longitude))
    dist = np.linalg.norm(_x-_y)
    
    mean_shift = MeanShift(bandwidth=dist).fit(X)
    coords = mean_shift.cluster_centers_
    return coords

In [10]:
def create_voronoi_cell(coords, hull):
    poly_shapes, pts, poly_to_pt_assignments = voronoi_regions_from_coords(coords, hull)
    cell_df = gpd.GeoDataFrame(poly_shapes, columns=['geometry'])
    cell_df = cell_df.reset_index().rename(columns={'index': 'cell_id'})
    return cell_df

In [11]:
X = stop_df[['lng', 'lat']]

#### Parameter settings
* k-means: [250, 200, 150, 100, 50]
* DBSCAN: [2, 3, 4, 5, 6]
* OPTICS: [-, 3, 4, 5, 6]
* Mean shift: [200, 400, 600, 800, 1000]

In [12]:
# K = 50
# coords = get_KMeans_cluster_centers(X, K)

minPts = 2
coords = get_DBSCAN_cluster_centers(X, minPts)

# minPts = 6
# coords = get_OPTICS_cluster_centers(X, minPts)

# bandwidth = 1000
# coords = get_MeanShift_cluster_centers(X, bandwidth)

In [13]:
hull = stop_gdf.geometry.unary_union.convex_hull.envelope

cell_df = create_voronoi_cell(coords, hull)
cell_df.head()

,cell_id,geometry
0,0,"POLYGON ((126.6534031 37.35929288611246, 126.8..."
1,1,"POLYGON ((126.6534031 36.64976795329063, 127.2..."
2,2,"POLYGON ((126.6534031 37.50447616178172, 126.8..."
3,3,"POLYGON ((127.2145628135683 36.78855536544284,..."
4,4,"POLYGON ((127.2145628135683 36.78855536544284,..."


In [14]:
def interior_point(points, polygon):
    sindex = points.sindex # r-tree spatial indexing
    possible_matches_index = list(sindex.intersection(polygon.bounds))
    possible_matches = points.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(polygon)]
    return precise_matches

In [15]:
def fixed_cell_mapping(points, cells):
    fixed_cell_df = pd.DataFrame()
    
    for i in tqdm(cells.index):
        if len(interior_point(points, cells.geometry[i])) != 0:
            cell = interior_point(points, cells.geometry[i])
            cell['cell_id'] = cells.cell_id[i]
            cell['cell_geometry'] = cells.geometry[i]
            fixed_cell_df = fixed_cell_df.append(cell)
            
    fixed_cell_df = fixed_cell_df.reset_index(drop=True)
    return fixed_cell_df

In [16]:
voronoi_cell_df = fixed_cell_mapping(stop_gdf, cell_df)

100%|████████████████████████████████████████████████████████████████████████████████| 117/117 [00:04<00:00, 23.59it/s]


In [17]:
def choropleth_map(cell_df):
    import json, warnings
    warnings.filterwarnings(action='ignore')
    
    sub_cell_df = cell_df.drop_duplicates(subset=['cell_id'])
    geo_json = json.loads(gpd.GeoSeries(sub_cell_df.cell_geometry).to_json())
    
    count_df = cell_df.copy()
    count_df['point'] = 1
    count_df = count_df.groupby(['cell_id']).count()[['point']].reset_index()
    
    for idx, obj in enumerate(geo_json['features']):
        obj['properties'].update({'cell_id': int(count_df.cell_id[idx])})
        
    osm_map = folium.Map(location=[37.5666103, 126.9783882], zoom_start=12)
    osm_map.choropleth(
        geo_data=geo_json,
        data=count_df,
        columns=['cell_id', 'point'],
        fill_color='YlGnBu',
        key_on='properties.cell_id',
        # threshold_scale=[0.0, 1.9, 12.0, 22.0, 33.0, 43.0, 54.0, 64.0],
        highligth=True,
        fill_opacity=0.75,
        line_opacity=0.5,
        legend_name='# of stop points'
    )
    return osm_map

In [18]:
choropleth_map(voronoi_cell_df)

### Feature extraction
* Activity Frequency  

$\begin{align}Pr(a_{i}=l|b_{i}):=\frac{\sum^{N}_{j=1}\delta_{a_{j},l}\cdot\delta_{b_{j},b_{i}}}
{\sum^{L}_{l=1}\sum^{N}_{j=1}\delta_{a_{j},l}\cdot\delta_{b_{j},b_{i}}}\end{align}$

In [19]:
def spatial_frequency(cells, activity_classes=4):
    column_names = ['cell_id', 'spatial_1', 'spatial_2', 'spatial_3', 'spatial_4']
    spatial_frequency = list()

    for cell_id in cells.cell_id.unique():
        activity_frequency = [cell_id]
        cell = cells[cells.cell_id == cell_id]
        activity_count = cell['cell_id'].groupby(cell['activity']).count()
        total = len(cell)

        for activity_type in range(activity_classes):
            if activity_type in cell.activity.unique():
                activity_frequency.append(activity_count[activity_type] / total)
            else:
                activity_frequency.append(0.0)
        spatial_frequency.append(activity_frequency)
    
    spatial_frequency_df = pd.DataFrame(spatial_frequency, columns=column_names)
    return spatial_frequency_df

In [20]:
activity_classes = len(stop_df.activity.unique())

In [21]:
spatial_frequency_df = spatial_frequency(voronoi_cell_df)
spatial_frequency_df = pd.merge(voronoi_cell_df, spatial_frequency_df, how='left')
spatial_frequency_df = spatial_frequency_df[['uid', 'id', 'spatial_1', 'spatial_2', 'spatial_3', 'spatial_4']]
spatial_frequency_df.head()

,uid,id,spatial_1,spatial_2,spatial_3,spatial_4
0,731,1568383,0.0,0.166667,0.166667,0.666667
1,3628,559217,0.0,0.166667,0.166667,0.666667
2,8141,887514,0.0,0.166667,0.166667,0.666667
3,864,1187347,0.0,0.166667,0.166667,0.666667
4,8466,1194465,0.0,0.166667,0.166667,0.666667


In [22]:
poi_df = pd.read_csv('./data/POIs_mapping.csv')
poi_gdf = df_to_gdf(poi_df, x='lng', y='lat')
poi_gdf['activity'] = poi_gdf['activity_class']

In [23]:
map_cell_df = voronoi_cell_df.drop_duplicates(subset=['cell_id'])[['cell_id', 'cell_geometry']]
map_cell_df = map_cell_df.rename(columns={'cell_geometry': 'geometry'})

In [24]:
voronoi_cell_poi_df = fixed_cell_mapping(poi_gdf, map_cell_df)

100%|████████████████████████████████████████████████████████████████████████████████| 117/117 [00:08<00:00, 17.03it/s]


In [25]:
contextual_frequency_df = spatial_frequency(voronoi_cell_poi_df)
contextual_frequency_df = contextual_frequency_df.rename(
    columns={'spatial_1': 'contextual_1',
             'spatial_2': 'contextual_2',
             'spatial_3': 'contextual_3',
             'spatial_4': 'contextual_4'})

In [26]:
uniform_prob = 1.0 / activity_classes
contextual_frequency_df = pd.merge(voronoi_cell_df, contextual_frequency_df, how='left')
contextual_frequency_df = contextual_frequency_df[['id', 'contextual_1', 'contextual_2', 'contextual_3', 'contextual_4']]
contextual_frequency_df = contextual_frequency_df.fillna(uniform_prob)
contextual_frequency_df.head()

,id,contextual_1,contextual_2,contextual_3,contextual_4
0,1568383,0.71087,0.002174,0.143478,0.143478
1,559217,0.71087,0.002174,0.143478,0.143478
2,887514,0.71087,0.002174,0.143478,0.143478
3,1187347,0.71087,0.002174,0.143478,0.143478
4,1194465,0.71087,0.002174,0.143478,0.143478


In [27]:
activity_frequency_df = pd.merge(spatial_frequency_df, contextual_frequency_df)
activity_frequency_df.head()

,uid,id,spatial_1,spatial_2,spatial_3,spatial_4,contextual_1,contextual_2,contextual_3,contextual_4
0,731,1568383,0.0,0.166667,0.166667,0.666667,0.71087,0.002174,0.143478,0.143478
1,3628,559217,0.0,0.166667,0.166667,0.666667,0.71087,0.002174,0.143478,0.143478
2,8141,887514,0.0,0.166667,0.166667,0.666667,0.71087,0.002174,0.143478,0.143478
3,864,1187347,0.0,0.166667,0.166667,0.666667,0.71087,0.002174,0.143478,0.143478
4,8466,1194465,0.0,0.166667,0.166667,0.666667,0.71087,0.002174,0.143478,0.143478


In [ ]:
assert len(stop_df) == len(activity_frequency_df)

* Distance-based empirical probability  

$\begin{align}Pr(a_{i}=l|\mathcal{X}_{l}(c_{i})):=\phi(d(p_{i},\mathcal{X}_{l}(c_{i})))\end{align}$  

and  

$\begin{align}Pr(a_{i}=l|\mathcal{A}_{l}(c_{i})):=\phi (d(p_{i},\mathcal{A}_{l}(c_{i})))\end{align}$

In [ ]:
def distance_based_probability(point_i, point_j, activity_classes):
    norm_dist_list = [point_i.id]
    
    for activity_type in range(activity_classes): 
        activity_cell = point_j[point_j['activity'] == activity_type]

        if len(activity_cell) != 0:
            sindex = activity_cell.sindex # r-tree
            nearest_index = list(sindex.nearest(point_i.geometry.bounds, 1))[0]
            dist = point_i.geometry.distance(activity_cell.iloc[nearest_index].geometry)
            norm_dist = (1 + (dist**2))**-1
            norm_dist_list.append(norm_dist)
        else:
            norm_dist_list.append(0.0)
    return norm_dist_list

In [ ]:
# Historical Neighbor Activity Confidence
def historical_neighbor(cell, activity_classes=4):
    column_names = ['id', 'historical_1 (min)', 'historical_2 (min)', 'historical_3 (min)', 'historical_4 (min)']
    historical_neighbor_df = pd.DataFrame()
    
    for i in cell.index:
        point_i = cell.loc[i]
        point_j = cell[~cell.index.isin([point_i.name])]
        dist_probs = distance_based_probability(point_i, point_j, activity_classes)
        historical_neighbor_df = historical_neighbor_df.append([dist_probs])
        
    historical_neighbor_df.columns = column_names
    return historical_neighbor_df

In [ ]:
# Contextual Neighbor Activity Confidence
def contextual_neighbor(cell, pois, activity_classes=4):
    column_names = ['id', 'contextual_1 (min)', 'contextual_2 (min)', 'contextual_3 (min)', 'contextual_4 (min)']
    contextual_neighbor_df = pd.DataFrame()
    
    for i in cell.index:
        point_i = cell.loc[i]
        point_j = pois[pois.cell_id == point_i.cell_id]
        
        if len(point_j) != 0:
            dist_probs = distance_based_probability(point_i, point_j, activity_classes)
        else:
            dist_probs = [point_i.id]
            zero_probs = [0.0] * activity_classes
            dist_probs.extend(zero_probs)
        contextual_neighbor_df = contextual_neighbor_df.append([dist_probs])
    
    contextual_neighbor_df.columns = column_names
    return contextual_neighbor_df

In [ ]:
historical_dist_probs_df = pd.DataFrame()
contextual_dist_probs_df = pd.DataFrame()

for cell_id in tqdm(voronoi_cell_df.cell_id.unique()):
    cell_i = voronoi_cell_df[voronoi_cell_df.cell_id == cell_id]
    
    historical_dist_probs = historical_neighbor(cell_i)
    contextual_dist_probs = contextual_neighbor(cell_i, voronoi_cell_poi_df)
    
    historical_dist_probs_df = historical_dist_probs_df.append(historical_dist_probs)
    contextual_dist_probs_df = contextual_dist_probs_df.append(contextual_dist_probs)

 79%|███████████████████████████████████████████████████████████████▋                 | 92/117 [00:17<00:04,  5.95it/s]

In [ ]:
dist_probs_df = pd.merge(historical_dist_probs_df, contextual_dist_probs_df)
dist_probs_df.head()

In [ ]:
assert len(stop_df) == len(dist_probs_df)

### Feature vector
$\begin{align}x=[Temporal\ Activity\ Frequency \in \mathbb{R}^{1 \times L},\end{align}$<br/>
$\begin{align}\qquad Spatial\ Activity\ Frequency \in \mathbb{R}^{1 \times L},\end{align}$<br/>
$\begin{align}\qquad Contextual\ Activity\ Frequency \in \mathbb{R}^{1 \times L},\end{align}$<br/>
$\begin{align}\qquad Historical\ Activity\ Confidence \in \mathbb{R}^{1 \times L},\end{align}$<br/>
$\begin{align}\qquad Contextual\ Activity\ Confidence \in \mathbb{R}^{1 \times L}]^{T} \in \mathbb{R}^{5L} \end{align}$

In [ ]:
feature_vector_df = pd.merge(activity_frequency_df, dist_probs_df, on='id')
feature_vector_df = pd.merge(feature_vector_df, stop_df[['uid', 'id', 'activity']]) # label
feature_vector_df.head()

In [ ]:
assert len(stop_df) == len(feature_vector_df)

### Save

In [ ]:
# feature_vector_df.to_csv(f'./data/parameter_settings/pre_voronoi_k-means_{K}.csv', index=False)
# feature_vector_df.to_csv(f'./data/parameter_settings/pre_voronoi_DBSCAN_{minPts}.csv', index=False)
# feature_vector_df.to_csv(f'./data/parameter_settings/pre_voronoi_OPTICS_{minPts}.csv', index=False)
# feature_vector_df.to_csv(f'./data/parameter_settings/pre_voronoi_MeanShift_{bandwidth}.csv', index=False)

In [ ]:
count_df = voronoi_cell_df.groupby('cell_id').count()
count_df = count_df.reset_index()[['cell_id', 'id']]
count_df = count_df[count_df.id == 1]

train_ids = voronoi_cell_df[voronoi_cell_df.cell_id.isin(count_df.cell_id.tolist())]
train_ids = train_ids[['id']]

In [ ]:
# train_ids.to_csv(f'./data/parameter_settings/pre_voronoi_k-means_{K}_train_ids.csv', index=False)
# train_ids.to_csv(f'./data/parameter_settings/pre_voronoi_MeanShift_{bandwidth}_train_ids.csv', index=False)